<a href="https://colab.research.google.com/github/glaiml/prj11-nlp-Project-1/blob/master/VGK_SeqNLP_Project1_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGK- IMDB-Sentiment Classification


## Loading the dataset

In [1]:
%tensorflow_version 2.x
from keras.datasets import imdb

#vocab_size = 10000 #vocab size

# (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

TensorFlow 2.x selected.


Using TensorFlow backend.


In [0]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split

1. Import test and train data (5 Marks)
2. Import the labels ( train and test) (5 Marks)

In [3]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

17465344/17464789 [==============================] - 1s 0us/step


In [7]:
print (x_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    1
  194 1153  194 8255   78  228    5    6 1463 4369 5012  134   26    4
  715    8  118 1634   14  394   20   13  119  954  189  102    5  207
  110 3103   21   14   69  188    8   30   23    7    4  249  126   93
    4  114    9 2300 1523    5  647    4  116    9   35 8163    4  229
    9  340 1322    4  118    9    4  130 4901   19    4 1002    5   89
   29  952   46   37    4  455    9   45   43   38 1543 1905  398    4
 1649 

In [8]:
print(y_train[1])

0


In [13]:
import numpy as np

unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[    0     1]
 [12500 12500]]


In [14]:
unique_elements, counts_elements = np.unique(y_test, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[    0     1]
 [12500 12500]]


## WORD INDEX BUILDING

3. Get the word index and then Create a key-value pair for word and word_id (12.5 points)

## Build Keras Embedding Layer Model

4. Build a Sequential Model using Keras for the Sentiment Classification task (10 points)

We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

## Model Accuracy
5. Report the Accuracy of the model (5 points)

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

6. Retrieve the output of each layer in Keras for a given single test sample from the trained model you built (2.5 points)